# Imports

In [1]:
import argparse
import os
import sys
import pickle
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# Reading Files

In [2]:
file_train = pd.read_csv('reviews_train.csv', dtype={'real review?': int,'category': str, 'rating': int, 'text_': str})
file_validation = pd.read_csv('reviews_validation.csv', dtype={'real review?': int,'category': str, 'rating': int, 'text_': str})

/home/sanumala/.conda/envs/cent7/5.1.0-py36/DPLClass/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
file_train

,real review?,category,rating,text_,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,1,Books_5,5,"I usually don't bother reviewing this book, bu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Clothing_Shoes_and_Jewelry_5,5,This cross is simple but elegant and just the ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Pet_Supplies_5,4,"From my experience,this stuff will last a long...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Kindle_Store_5,4,the mysteries that involved a young woman who ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Books_5,3,Why do these entertain me? The fact that they...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37179,1,Clothing_Shoes_and_Jewelry_5,4,"They were a little large, but I thought they w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37180,1,Tools_and_Home_Improvement_5,5,"Just as nice as the old ones, the finish is a ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37181,0,Toys_and_Games_5,3,This is probably entertaining for a baby who c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37182,0,Clothing_Shoes_and_Jewelry_5,5,"Had to have this color for work, and they were...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Extracting features from text column

In [5]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range = (1,2))
        # Transform text data to list of strings
corpora = file_train['text_'].astype(str).values.tolist()    # Obtain featurizer from data
vectorizer.fit(corpora)
        # Create feature vector
X = vectorizer.transform(corpora)
print(X.shape[0])

37184


In [6]:
#get the column names
vocab = vectorizer.get_feature_names()

# Transforming the columns

In [7]:

categoryFeatures = pd.get_dummies(file_train['category'])
ratingFeatures = pd.get_dummies(file_train['rating'])

df = pd.DataFrame(categoryFeatures)
df = df.join(ratingFeatures)


In [8]:
#changing the name to make it clear
train_df = df
train_df

,Books_5,Clothing_Shoes_and_Jewelry_5,Electronics_5,Home_and_Kitchen_5,Kindle_Store_5,Movies_and_TV_5,Pet_Supplies_5,Sports_and_Outdoors_5,Tools_and_Home_Improvement_5,Toys_and_Games_5,1,2,3,4,5
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37179,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
37180,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
37181,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
37182,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [9]:
X1 = pd.DataFrame(X.toarray(), columns = vocab)
X1 = X1.join(train_df)

In [16]:
X1

,00,10,10 and,100,11,12,13,14,15,16,...,Movies_and_TV_5,Pet_Supplies_5,Sports_and_Outdoors_5,Tools_and_Home_Improvement_5,Toys_and_Games_5,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,1,0,0,0,0,0,0,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,1,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,1,0
37180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,0,0,1,0,0,0,0,0,1
37181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.074983,0.083512,...,0,0,0,0,1,0,0,1,0,0
37182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1


# Model Creation

### Training

In [20]:
#Initializing the Logistic Regression
classifier = LogisticRegression()
classifier.fit(X1 , file_train['real review?'])

/home/sanumala/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [21]:
y_pred = classifier.predict(X1)

In [22]:
# y_pred = clf.predict(X1)
accuracy = (y_pred == file_train['real review?']).sum() / file_train['real review?'].size
accuracy

0.9458907056798623

# Validating

In [23]:
categoryFeatures = pd.get_dummies(file_validation['category'])
ratingFeatures = pd.get_dummies(file_validation['rating'])

df = pd.DataFrame(categoryFeatures)
df = df.join(ratingFeatures)

In [24]:
corpora = file_validation['text_'].astype(str).values.tolist()
        # Obtain featurizer from data
Xvalid = vectorizer.transform(corpora)

In [25]:
valid_df = df

In [26]:
X2 = pd.DataFrame(Xvalid.toarray(), columns = vocab)
X2 = X2.join(valid_df)

In [27]:
y_pred = classifier.predict(X2)

In [28]:
accuracy = (y_pred == file_validation['real review?']).sum() / file_validation['real review?'].size

In [29]:
accuracy

0.934934934934935

# Testing

In [30]:
file_test = pd.read_csv('reviews_test_attributes.csv' ,dtype={'real review?': int,'category': str, 'rating': int, 'text_': str})
corpora = file_test['text_'].astype(str).values.tolist()
Xtest = vectorizer.transform(corpora)

In [31]:
categoryFeatures = pd.get_dummies(file_test['category'])
ratingFeatures = pd.get_dummies(file_test['rating'])

df = pd.DataFrame(categoryFeatures)
df = df.join(ratingFeatures)

In [32]:
X3 = pd.DataFrame(Xtest.toarray(), columns = vocab)
X3 = X3.join(df)

X3

In [34]:
y_hat = classifier.predict_proba(X3)
y_hat[:,1]

array([0.9195009 , 0.97601451, 0.02893975, ..., 0.5488467 , 0.37898862,
       0.05113153])

In [36]:
y_pred = list(y_hat[:,1])
x_column = list(file_test.index)
df = pd.DataFrame(x_column, columns = ['ID'])
df['real review?'] = y_pred
df.to_csv('predictionsSVM.csv', index=False)

# To conclude: recieved an accuracy of 0.98354 trying to predict if the review is fake or not. (Ranked Top 13 in Kaggle Competition)